In [ ]:
# Fetch all .po files from FE Workflows (for smart start) - Extract copy and create one giant .csv with the translations

In [ ]:
! pip install selenium

In [2]:
# import libraries

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from random import randint
import datetime
import os
import time
import pandas as pd

In [3]:
url = "http://fe.smartstart.prd.feersum.io/admin/workflow/workflow/"

In [4]:
# create a new Firefox session
driver = webdriver.Firefox()
# driver.implicitly_wait(30)
driver.get(url)

In [5]:
# Fill in login details
username = driver.find_element_by_id("id_username")
password = driver.find_element_by_id("id_password")

username.send_keys("devs")
password.send_keys("axvkcl4389xv%^3klj498dfg")

In [6]:
#Login
python_button = driver.find_element_by_class_name("btn ")
python_button.click() 

In [7]:
# Get page source for first page of interest
main_soup=BeautifulSoup(driver.page_source, 'lxml')

In [16]:
# determine total number of workflows
number_of_workflows = int(main_soup.find('div',class_="pagination-info muted").text.split()[-3])

In [24]:
def page_data():
    """ fetch all the .po links on a page
    """
    #Selenium hands the page source to Beautiful Soup
    inner_soup=BeautifulSoup(driver.page_source, 'lxml')

    for link in inner_soup.find_all('button', string="Export PO"):
        workflow_links.append(link)

In [22]:
workflow_links = [] #empty list
work_flow_counter = 0
page_index = 2

while work_flow_counter < number_of_workflows:
    page_data()
    page_index+=1
    work_flow_counter = len(workflow_links)
    python_button = driver.find_element_by_link_text(str(page_index))
    python_button.click() 

In [ ]:
number_of_workflows = 